In [1]:
import glob
import pandas as pd
import numpy as np

import h2o
from h2o.automl import H2OAutoML

In [2]:
df = pd.concat([pd.read_csv(f, encoding='latin1') for f in glob.glob('data/final/*.csv')])

In [3]:
df.to_pickle('data/final/df.pkl')

In [4]:
df.shape

(4339196, 33)

In [5]:
h2o.init(nthreads=-1, max_mem_size='10G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_252"; OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~deb9u1-b09); OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
  Starting server from /home/jupyter/.local/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqrwjpp7k
  JVM stdout: /tmp/tmpqrwjpp7k/h2o_jupyter_started_from_python.out
  JVM stderr: /tmp/tmpqrwjpp7k/h2o_jupyter_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,7 months and 29 days !!!
H2O cluster name:,H2O_from_python_jupyter_xl2t1v
H2O cluster total nodes:,1
H2O cluster free memory:,8.89 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [6]:
hf = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
# cluster
# origin_day, origin_hour, origin_density_cluster, origin_speed_cluster, dest_density_cluster, dest_speed_cluster, 
for c in [
    "origin_day", "origin_hour", "origin_density_cluster", "origin_speed_cluster", "dest_density_cluster", "dest_speed_cluster",
    "day_hour_crossing", "origin_dest_density_cluster_crossing", "origin_dest_speed_cluster_crossing"]:
    hf[c] = hf[c].asfactor()

In [8]:
features = [
    'origin_day', 'origin_hour', 'origin_day_sin', 'origin_day_cos', 'origin_hour_sin', 'origin_hour_cos',
    'origin_density_cluster', 'origin_speed_cluster',
    'origin_distance_from_density_center', 'origin_bearing_from_density_center',
    'origin_distance_from_speed_center', 'origin_bearing_from_speed_center',
    'dest_density_cluster', 'dest_speed_cluster',
    'dest_distance_from_density_center', 'dest_bearing_from_density_center',
    'dest_distance_from_speed_center', 'dest_bearing_from_speed_center',
    'distance_origin_dest', 'bearing_origin_dest',
    'distance_origin_dest_density_cluster', 'distance_origin_dest_speed_cluster',
    'origin_distance_from_density_center_squared', 'origin_distance_from_speed_center_squared',
    'dest_distance_from_density_center_squared', 'dest_distance_from_speed_center_squared',
    'distance_origin_dest_squared', 
    'distance_origin_dest_density_cluster_squared', 'distance_origin_dest_speed_cluster_squared',
    'day_hour_crossing', 
    'origin_dest_density_cluster_crossing', 'origin_dest_speed_cluster_crossing',
]
label = 'duration'

In [9]:
aml = H2OAutoML(max_models=50, seed=67, max_runtime_secs=7200)
aml.train(x=features, y=label, training_frame=hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [10]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

This H2OFrame is empty.


In [11]:
aml.leader